In [1]:
#LSTM


In [2]:
import pandas as pd

# Read the data set
data = pd.read_csv("new_df_selected3_last3years_adjusted.csv")


unique_company_names = data['company_name'].nunique()
unique_status_labels = data['status_label'].nunique()
unique_divisions = data['Division'].nunique()
unique_majorgroup = data['MajorGroup'].nunique()
unique_last_year = data['last_year'].nunique()

print("Number of unique values in 'company_name' column:", unique_company_names)
print("Number of unique values in 'status_label' column:", unique_status_labels)
print("Number of unique values in 'Division' column:", unique_divisions)
print("Number of unique values in 'MajorGroup' column:", unique_majorgroup)
print("Number of unique values in 'last_year' column:", unique_last_year)



Number of unique values in 'company_name' column: 8971
Number of unique values in 'status_label' column: 2
Number of unique values in 'Division' column: 10
Number of unique values in 'MajorGroup' column: 73
Number of unique values in 'last_year' column: 20


In [3]:
data.shape

(8971, 119)

In [4]:
# Encoding non-numeric columns

from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder object
label_encoder = LabelEncoder()


# Label-encode the company_name column
data['company_name_encoded'] = label_encoder.fit_transform(data['company_name'])


# Label-encode the Division column
data['Division_encoded'] = label_encoder.fit_transform(data['Division'])

# Label-encode MajorGroup columns
data['MajorGroup_encoded'] = label_encoder.fit_transform(data['MajorGroup'])
#When using label encoding for feature encoding, the sequential relationship between categories will not be introduced and will not have an impact on prediction.

# Encode the label of the status_label column
data['status_label_encoded'] = label_encoder.fit_transform(data['status_label'])
#With only two categories, it may be simpler and more appropriate to use label encoding as it maps the categories to 0 and 1, suitable for use in tree-based models. 
#If use one-hot encoding, a new column will be generated

print(data.head())


D:\Anaconda\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


  company_name status_label Division  MajorGroup  last_year  X1_last1year  \
0          C_1        alive        D          37     2017.0         942.7   
1          C_2        alive        D          36     2010.0        1107.7   
2          C_3        alive        D          38     2008.0       12686.0   
3          C_4        alive        D          28     2007.0      581502.0   
4          C_5        alive        D          35     1999.0       28957.0   

   X1_last2year  X1_last3year  X2_last1year  X2_last2year  ...  \
0         888.5         873.1       1524.70        1504.1  ...   
1         900.2        1077.4       1474.50        1343.6  ...   
2       13454.0       13582.0      21401.00       27171.0  ...   
3      353541.0     1037047.0    1288165.00      927239.0  ...   
4           NaN           NaN         42.21           NaN  ...   

   nyse_last1year  nyse_last2year  nyse_last3year  nasdaq_last1year  \
0    11912.848307    10451.377523    10606.906738       6293.024211  

In [5]:
data.shape

(8971, 123)

In [6]:
missing_rows_count = data.isnull().any(axis=1).sum()
print("Number of rows with missing values:", missing_rows_count)


Number of rows with missing values: 2673


In [7]:
# Delete rows with missing values
data_cleaned = data.dropna()
# Delete non-numeric columns that are not encoded
# Delete specified column
data_cleaned = data_cleaned.drop(['company_name', 'status_label', 'Division', 'MajorGroup', 'last_year', 'company_name_encoded'], axis=1)

data_cleaned.shape

(6298, 117)

In [8]:
data_cleaned.head

<bound method NDFrame.head of       X1_last1year  X1_last2year  X1_last3year  X2_last1year  X2_last2year  \
0            942.7         888.5         873.1       1524.70       1504.10   
1           1107.7         900.2        1077.4       1474.50       1343.60   
2          12686.0       13454.0       13582.0      21401.00      27171.00   
3         581502.0      353541.0     1037047.0    1288165.00     927239.00   
5           6838.0        6642.0        5935.0      25088.00      25438.00   
...            ...           ...           ...           ...           ...   
8966       10566.0       11738.0        9599.0      28278.00      26206.00   
8967        3369.0        9049.0       21381.0       3466.00       9198.00   
8968        2482.2        2340.6        2071.2       9401.50      10252.40   
8969         931.6        1032.7         829.3       2810.20       2542.00   
8970       82589.0      135207.0       63971.0       1625.37       1736.11   

      X2_last3year  X3_last1year 

In [9]:
status_counts = data_cleaned['status_label_encoded'].value_counts()
print(status_counts)


0    5784
1     514
Name: status_label_encoded, dtype: int64


In [10]:
#the impact of imbalanced datasets

### 1.1 imbalance dataset

In [11]:
# Feature column names
last1year_features = []
last2year_features = []
last3year_features = []

# Loop to build list of feature column names
for i in range(1, 19):
    last1year_features.append(f'X{i}_last1year')
    last1year_features.append(f'X{i}_last1year_ycr')
    last2year_features.append(f'X{i}_last2year')
    last2year_features.append(f'X{i}_last2year_ycr')
    last3year_features.append(f'X{i}_last3year')
    last3year_features.append(f'X{i}_last3year_ycr')

last1year_features.extend(['nyse_last1year', 'nasdaq_last1year', 'Division_encoded', 'MajorGroup_encoded'])
last2year_features.extend(['nyse_last2year', 'nasdaq_last2year', 'Division_encoded', 'MajorGroup_encoded'])
last3year_features.extend(['nyse_last3year', 'nasdaq_last3year', 'Division_encoded', 'MajorGroup_encoded'])

# Define target column name
target_column = 'status_label_encoded'

# Extract features and target
X_last1year = data_cleaned[last1year_features].values
X_last2year = data_cleaned[last2year_features].values
X_last3year = data_cleaned[last3year_features].values
y = data_cleaned[target_column].values

# Print data structure
print("Shape of X_last1year:", X_last1year.shape)
print("Shape of X_last2year:", X_last2year.shape)
print("Shape of X_last3year:", X_last3year.shape)
print("Shape of y:", y.shape)



# Data Standardization
import numpy as np
from sklearn.preprocessing import StandardScaler

# Define the scaler
scaler = StandardScaler()
# Fit the scaler on the entire data set
scaler.fit(np.concatenate((X_last1year, X_last2year, X_last3year,), axis=0))
# Transform each year's data using the same scaler
X_last1year_scaled = scaler.transform(X_last1year)
X_last2year_scaled = scaler.transform(X_last2year)
X_last3year_scaled = scaler.transform(X_last3year)

# Reshape features to 3D arrays [samples, time steps, features]
X_last1year_reshaped = np.reshape(X_last1year_scaled, (X_last1year_scaled.shape[0], 1, X_last1year_scaled.shape[1]))
X_last2year_reshaped = np.reshape(X_last2year_scaled, (X_last2year_scaled.shape[0], 1, X_last2year_scaled.shape[1]))
X_last3year_reshaped = np.reshape(X_last3year_scaled, (X_last3year_scaled.shape[0], 1, X_last3year_scaled.shape[1]))

# Combine last1year and last2year features along the time step dimension
X_combined_reshaped = np.concatenate((X_last3year_reshaped, X_last2year_reshaped, X_last1year_reshaped), axis=1)
#First use data from the past two years (X_last3year_reshaped), then data from the past years 

# Print data structure
print("Shape of X_last1year_scaled:", X_last1year_scaled.shape)
print("Shape of X_last2year_scaled:", X_last2year_scaled.shape)
print("Shape of X_last3year_scaled:", X_last3year_scaled.shape)
print("Shape of X_combined_reshaped:", X_combined_reshaped.shape)



# Split into train, validation, and test sets
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(X_combined_reshaped, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42) # 0.8 * 0.25 = 0.2

print("Shape of X_train:", X_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of X_test:", X_test.shape)




from keras.layers import LSTM, Dense, Dropout, Input
from keras.models import Model
from keras.optimizers import Adam

# Define LSTM model
inputs = Input(shape=(3, X_train.shape[2]))  # (2, number of features) Accepts data with two time steps as input, and the first time step is the data of the past two years and the second time step is the data of the past year.
lstm_layer = LSTM(100)(inputs)
dropout_layer = Dropout(0.2)(lstm_layer)
output_layer = Dense(1, activation='sigmoid')(dropout_layer)

model = Model(inputs=inputs, outputs=output_layer)

# Compile model
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_val, y_val), verbose=2)

## Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print("test loss:", loss)
print("Test accuracy:", accuracy)

Shape of X_last1year: (6298, 40)
Shape of X_last2year: (6298, 40)
Shape of X_last3year: (6298, 40)
Shape of y: (6298,)
Shape of X_last1year_scaled: (6298, 40)
Shape of X_last2year_scaled: (6298, 40)
Shape of X_last3year_scaled: (6298, 40)
Shape of X_combined_reshaped: (6298, 3, 40)
Shape of X_train: (3778, 3, 40)
Shape of X_val: (1260, 3, 40)
Shape of X_test: (1260, 3, 40)
Epoch 1/100
237/237 - 3s - 15ms/step - accuracy: 0.8941 - loss: 0.3255 - val_accuracy: 0.9310 - val_loss: 0.2239
Epoch 2/100
237/237 - 1s - 4ms/step - accuracy: 0.9161 - loss: 0.2493 - val_accuracy: 0.9310 - val_loss: 0.2194
Epoch 3/100
237/237 - 1s - 3ms/step - accuracy: 0.9179 - loss: 0.2403 - val_accuracy: 0.9325 - val_loss: 0.2139
Epoch 4/100
237/237 - 1s - 3ms/step - accuracy: 0.9177 - loss: 0.2295 - val_accuracy: 0.9310 - val_loss: 0.2101
Epoch 5/100
237/237 - 1s - 4ms/step - accuracy: 0.9203 - loss: 0.2251 - val_accuracy: 0.9286 - val_loss: 0.2141
Epoch 6/100
237/237 - 1s - 4ms/step - accuracy: 0.9211 - loss: 

In [12]:
# Make predictions on the test set
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate evaluation metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Calculate precision
precision = precision_score(y_test, y_pred)

# Calculate recall
recall = recall_score(y_test, y_pred)

# Calculate F1 score
f1 = f1_score(y_test, y_pred)

# Calculate micro F1 score
micro_f1 = f1_score(y_test, y_pred, average='micro')

# Calculate macro F1 score
macro_f1 = f1_score(y_test, y_pred, average='macro')

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_test, y_pred_prob)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation metrics
print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("F1 Score:", f1)
print("Micro F1 Score:", micro_f1)
print("Macro F1 Score:", macro_f1)
print("ROC AUC:", roc_auc)
print("Confusion Matrix:")
print(conf_matrix)


40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
Accuracy: 0.9007936507936508
Recall: 0.21428571428571427
Precision: 0.30434782608695654
F1 Score: 0.25149700598802394
Micro F1 Score: 0.9007936507936508
Macro F1 Score: 0.5991866670399109
ROC AUC: 0.7510801222382241
Confusion Matrix:
[[1114   48]
 [  77   21]]


### 1.2 SMOTE

In [13]:
# Feature column names
last1year_features = []
last2year_features = []
last3year_features = []

# Loop to build list of feature column names
for i in range(1, 19):
    last1year_features.append(f'X{i}_last1year')
    last1year_features.append(f'X{i}_last1year_ycr')
    last2year_features.append(f'X{i}_last2year')
    last2year_features.append(f'X{i}_last2year_ycr')
    last3year_features.append(f'X{i}_last3year')
    last3year_features.append(f'X{i}_last3year_ycr')

last1year_features.extend(['nyse_last1year', 'nasdaq_last1year', 'Division_encoded', 'MajorGroup_encoded'])
last2year_features.extend(['nyse_last2year', 'nasdaq_last2year', 'Division_encoded', 'MajorGroup_encoded'])
last3year_features.extend(['nyse_last3year', 'nasdaq_last3year', 'Division_encoded', 'MajorGroup_encoded'])

# Define target column name
target_column = 'status_label_encoded'

# Extract features and target
X_last1year = data_cleaned[last1year_features].values
X_last2year = data_cleaned[last2year_features].values
X_last3year = data_cleaned[last3year_features].values
y = data_cleaned[target_column].values

# Print data structure
print("Shape of X_last1year:", X_last1year.shape)
print("Shape of X_last2year:", X_last2year.shape)
print("Shape of X_last3year:", X_last3year.shape)
print("Shape of y:", y.shape)



# Data Standardization
import numpy as np
from sklearn.preprocessing import StandardScaler

# Define the scaler
scaler = StandardScaler()
# Fit the scaler on the entire data set
scaler.fit(np.concatenate((X_last1year, X_last2year, X_last3year,), axis=0))
# Transform each year's data using the same scaler
X_last1year_scaled = scaler.transform(X_last1year)
X_last2year_scaled = scaler.transform(X_last2year)
X_last3year_scaled = scaler.transform(X_last3year)

# Reshape features to 3D arrays [samples, time steps, features]
X_last1year_reshaped = np.reshape(X_last1year_scaled, (X_last1year_scaled.shape[0], 1, X_last1year_scaled.shape[1]))
X_last2year_reshaped = np.reshape(X_last2year_scaled, (X_last2year_scaled.shape[0], 1, X_last2year_scaled.shape[1]))
X_last3year_reshaped = np.reshape(X_last3year_scaled, (X_last3year_scaled.shape[0], 1, X_last3year_scaled.shape[1]))

# Combine last1year and last2year features along the time step dimension
X_combined_reshaped = np.concatenate((X_last3year_reshaped, X_last2year_reshaped, X_last1year_reshaped), axis=1)
#First use data from the past two years (X_last3year_reshaped), then data from the past years 

# Print data structure
print("Shape of X_last1year_scaled:", X_last1year_scaled.shape)
print("Shape of X_last2year_scaled:", X_last2year_scaled.shape)
print("Shape of X_last3year_scaled:", X_last3year_scaled.shape)
print("Shape of X_combined_reshaped:", X_combined_reshaped.shape)



# Split into train, validation, and test sets
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(X_combined_reshaped, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42) # 0.8 * 0.25 = 0.2

print("Shape of X_train:", X_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of X_test:", X_test.shape)



from imblearn.over_sampling import SMOTE
# Flatten X_train
X_train_flat = X_train.reshape(X_train.shape[0], -1)
# Apply SMOTE only on the training set
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_flat, y_train)
print("Shape of X_train_resampled:", X_train_resampled.shape)
# Reshape resampled training data to match LSTM input shape
X_train_resampled = X_train_resampled.reshape(-1, 3, X_train.shape[2])
# Check the shape of resampled data and target variable
print("Shape of X_train_resampled:", X_train_resampled.shape)



from keras.layers import LSTM, Dense, Dropout, Input
from keras.models import Model
from keras.optimizers import Adam

# Define LSTM model
inputs = Input(shape=(3, X_train_resampled.shape[2]))  # (2, number of features) Accepts data with two time steps as input, and the first time step is the data of the past two years and the second time step is the data of the past year.
lstm_layer = LSTM(100)(inputs)
dropout_layer = Dropout(0.2)(lstm_layer)
output_layer = Dense(1, activation='sigmoid')(dropout_layer)

model = Model(inputs=inputs, outputs=output_layer)

# Compile model
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
history = model.fit(X_train_resampled, y_train_resampled, epochs=100, batch_size=16, validation_data=(X_val, y_val), verbose=2)

## Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print("test loss:", loss)
print("Test accuracy:", accuracy)

Shape of X_last1year: (6298, 40)
Shape of X_last2year: (6298, 40)
Shape of X_last3year: (6298, 40)
Shape of y: (6298,)
Shape of X_last1year_scaled: (6298, 40)
Shape of X_last2year_scaled: (6298, 40)
Shape of X_last3year_scaled: (6298, 40)
Shape of X_combined_reshaped: (6298, 3, 40)
Shape of X_train: (3778, 3, 40)
Shape of X_val: (1260, 3, 40)
Shape of X_test: (1260, 3, 40)
Shape of X_train_resampled: (6898, 120)
Shape of X_train_resampled: (6898, 3, 40)
Epoch 1/100
432/432 - 3s - 8ms/step - accuracy: 0.7231 - loss: 0.5310 - val_accuracy: 0.6127 - val_loss: 0.6013
Epoch 2/100
432/432 - 1s - 3ms/step - accuracy: 0.7925 - loss: 0.4376 - val_accuracy: 0.7698 - val_loss: 0.4359
Epoch 3/100
432/432 - 1s - 3ms/step - accuracy: 0.8231 - loss: 0.3874 - val_accuracy: 0.7286 - val_loss: 0.5113
Epoch 4/100
432/432 - 1s - 3ms/step - accuracy: 0.8444 - loss: 0.3459 - val_accuracy: 0.7929 - val_loss: 0.4134
Epoch 5/100
432/432 - 1s - 3ms/step - accuracy: 0.8600 - loss: 0.3181 - val_accuracy: 0.8175 -

In [14]:
# Make predictions on the test set
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate evaluation metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Calculate precision
precision = precision_score(y_test, y_pred)

# Calculate recall
recall = recall_score(y_test, y_pred)

# Calculate F1 score
f1 = f1_score(y_test, y_pred)

# Calculate micro F1 score
micro_f1 = f1_score(y_test, y_pred, average='micro')

# Calculate macro F1 score
macro_f1 = f1_score(y_test, y_pred, average='macro')

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_test, y_pred_prob)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation metrics
print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("F1 Score:", f1)
print("Micro F1 Score:", micro_f1)
print("Macro F1 Score:", macro_f1)
print("ROC AUC:", roc_auc)
print("Confusion Matrix:")
print(conf_matrix)


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Accuracy: 0.8896825396825396
Recall: 0.2857142857142857
Precision: 0.28865979381443296
F1 Score: 0.28717948717948716
Micro F1 Score: 0.8896825396825395
Macro F1 Score: 0.613697270471464
ROC AUC: 0.7741490744318381
Confusion Matrix:
[[1093   69]
 [  70   28]]


### 1.3  Undersampling

In [15]:
# Feature column names
last1year_features = []
last2year_features = []
last3year_features = []

# Loop to build list of feature column names
for i in range(1, 19):
    last1year_features.append(f'X{i}_last1year')
    last1year_features.append(f'X{i}_last1year_ycr')
    last2year_features.append(f'X{i}_last2year')
    last2year_features.append(f'X{i}_last2year_ycr')
    last3year_features.append(f'X{i}_last3year')
    last3year_features.append(f'X{i}_last3year_ycr')

last1year_features.extend(['nyse_last1year', 'nasdaq_last1year', 'Division_encoded', 'MajorGroup_encoded'])
last2year_features.extend(['nyse_last2year', 'nasdaq_last2year', 'Division_encoded', 'MajorGroup_encoded'])
last3year_features.extend(['nyse_last3year', 'nasdaq_last3year', 'Division_encoded', 'MajorGroup_encoded'])

# Define target column name
target_column = 'status_label_encoded'

# Extract features and target
X_last1year = data_cleaned[last1year_features].values
X_last2year = data_cleaned[last2year_features].values
X_last3year = data_cleaned[last3year_features].values
y = data_cleaned[target_column].values

# Print data structure
print("Shape of X_last1year:", X_last1year.shape)
print("Shape of X_last2year:", X_last2year.shape)
print("Shape of X_last3year:", X_last3year.shape)
print("Shape of y:", y.shape)



# Data Standardization
import numpy as np
from sklearn.preprocessing import StandardScaler

# Define the scaler
scaler = StandardScaler()
# Fit the scaler on the entire data set
scaler.fit(np.concatenate((X_last1year, X_last2year, X_last3year,), axis=0))
# Transform each year's data using the same scaler
X_last1year_scaled = scaler.transform(X_last1year)
X_last2year_scaled = scaler.transform(X_last2year)
X_last3year_scaled = scaler.transform(X_last3year)

# Reshape features to 3D arrays [samples, time steps, features]
X_last1year_reshaped = np.reshape(X_last1year_scaled, (X_last1year_scaled.shape[0], 1, X_last1year_scaled.shape[1]))
X_last2year_reshaped = np.reshape(X_last2year_scaled, (X_last2year_scaled.shape[0], 1, X_last2year_scaled.shape[1]))
X_last3year_reshaped = np.reshape(X_last3year_scaled, (X_last3year_scaled.shape[0], 1, X_last3year_scaled.shape[1]))

# Combine last1year and last2year features along the time step dimension
X_combined_reshaped = np.concatenate((X_last3year_reshaped, X_last2year_reshaped, X_last1year_reshaped), axis=1)
#First use data from the past two years (X_last3year_reshaped), then data from the past years 

# Print data structure
print("Shape of X_last1year_scaled:", X_last1year_scaled.shape)
print("Shape of X_last2year_scaled:", X_last2year_scaled.shape)
print("Shape of X_last3year_scaled:", X_last3year_scaled.shape)
print("Shape of X_combined_reshaped:", X_combined_reshaped.shape)



# Split into train, validation, and test sets
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(X_combined_reshaped, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42) # 0.8 * 0.25 = 0.2

print("Shape of X_train:", X_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of X_test:", X_test.shape)



from imblearn.under_sampling import RandomUnderSampler

# Flatten X_train
X_train_flat = X_train.reshape(X_train.shape[0], -1)
# Apply undersampling only on the training set
undersampler = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train_flat, y_train)
print("Shape of X_train_resampled:", X_train_resampled.shape)
# Reshape resampled training data to match LSTM input shape
X_train_resampled = X_train_resampled.reshape(-1, 3, X_train.shape[2])
# Check the shape of resampled data and target variable
print("Shape of X_train_resampled:", X_train_resampled.shape)




from keras.layers import LSTM, Dense, Dropout, Input
from keras.models import Model
from keras.optimizers import Adam

# Define LSTM model
inputs = Input(shape=(3, X_train_resampled.shape[2]))  # (2, number of features) Accepts data with two time steps as input, and the first time step is the data of the past two years and the second time step is the data of the past year.
lstm_layer = LSTM(100)(inputs)
dropout_layer = Dropout(0.2)(lstm_layer)
output_layer = Dense(1, activation='sigmoid')(dropout_layer)

model = Model(inputs=inputs, outputs=output_layer)

# Compile model
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
history = model.fit(X_train_resampled, y_train_resampled, epochs=100, batch_size=16, validation_data=(X_val, y_val), verbose=2)

## Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print("test loss:", loss)
print("Test accuracy:", accuracy)

Shape of X_last1year: (6298, 40)
Shape of X_last2year: (6298, 40)
Shape of X_last3year: (6298, 40)
Shape of y: (6298,)
Shape of X_last1year_scaled: (6298, 40)
Shape of X_last2year_scaled: (6298, 40)
Shape of X_last3year_scaled: (6298, 40)
Shape of X_combined_reshaped: (6298, 3, 40)
Shape of X_train: (3778, 3, 40)
Shape of X_val: (1260, 3, 40)
Shape of X_test: (1260, 3, 40)
Shape of X_train_resampled: (658, 120)
Shape of X_train_resampled: (658, 3, 40)
Epoch 1/100
42/42 - 2s - 57ms/step - accuracy: 0.6596 - loss: 0.6398 - val_accuracy: 0.5675 - val_loss: 0.6853
Epoch 2/100
42/42 - 0s - 7ms/step - accuracy: 0.7249 - loss: 0.5628 - val_accuracy: 0.6087 - val_loss: 0.6030
Epoch 3/100
42/42 - 0s - 7ms/step - accuracy: 0.7310 - loss: 0.5320 - val_accuracy: 0.6040 - val_loss: 0.5877
Epoch 4/100
42/42 - 0s - 7ms/step - accuracy: 0.7477 - loss: 0.5146 - val_accuracy: 0.6119 - val_loss: 0.5932
Epoch 5/100
42/42 - 0s - 7ms/step - accuracy: 0.7401 - loss: 0.4993 - val_accuracy: 0.6183 - val_loss: 

In [16]:
# Make predictions on the test set
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate evaluation metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Calculate precision
precision = precision_score(y_test, y_pred)

# Calculate recall
recall = recall_score(y_test, y_pred)

# Calculate F1 score
f1 = f1_score(y_test, y_pred)

# Calculate micro F1 score
micro_f1 = f1_score(y_test, y_pred, average='micro')

# Calculate macro F1 score
macro_f1 = f1_score(y_test, y_pred, average='macro')

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_test, y_pred_prob)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation metrics
print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("F1 Score:", f1)
print("Micro F1 Score:", micro_f1)
print("Macro F1 Score:", macro_f1)
print("ROC AUC:", roc_auc)
print("Confusion Matrix:")
print(conf_matrix)


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Accuracy: 0.7182539682539683
Recall: 0.6326530612244898
Precision: 0.16272965879265092
F1 Score: 0.2588726513569937
Micro F1 Score: 0.7182539682539683
Macro F1 Score: 0.5424691527240628
ROC AUC: 0.7575828093715972
Confusion Matrix:
[[843 319]
 [ 36  62]]


## undersampling, CNN-LSTM

In [17]:
# Feature column names
last1year_features = []
last2year_features = []
last3year_features = []

# Loop to build list of feature column names
for i in range(1, 19):
    last1year_features.append(f'X{i}_last1year')
    last1year_features.append(f'X{i}_last1year_ycr')
    last2year_features.append(f'X{i}_last2year')
    last2year_features.append(f'X{i}_last2year_ycr')
    last3year_features.append(f'X{i}_last3year')
    last3year_features.append(f'X{i}_last3year_ycr')

last1year_features.extend(['nyse_last1year', 'nasdaq_last1year', 'Division_encoded', 'MajorGroup_encoded'])
last2year_features.extend(['nyse_last2year', 'nasdaq_last2year', 'Division_encoded', 'MajorGroup_encoded'])
last3year_features.extend(['nyse_last3year', 'nasdaq_last3year', 'Division_encoded', 'MajorGroup_encoded'])

# Define target column name
target_column = 'status_label_encoded'

# Extract features and target
X_last1year = data_cleaned[last1year_features].values
X_last2year = data_cleaned[last2year_features].values
X_last3year = data_cleaned[last3year_features].values
y = data_cleaned[target_column].values

# Print data structure
print("Shape of X_last1year:", X_last1year.shape)
print("Shape of X_last2year:", X_last2year.shape)
print("Shape of X_last3year:", X_last3year.shape)
print("Shape of y:", y.shape)



# Data Standardization
import numpy as np
from sklearn.preprocessing import StandardScaler

# Define the scaler
scaler = StandardScaler()
# Fit the scaler on the entire data set
scaler.fit(np.concatenate((X_last1year, X_last2year, X_last3year,), axis=0))
# Transform each year's data using the same scaler
X_last1year_scaled = scaler.transform(X_last1year)
X_last2year_scaled = scaler.transform(X_last2year)
X_last3year_scaled = scaler.transform(X_last3year)

# Reshape features to 3D arrays [samples, time steps, features]
X_last1year_reshaped = np.reshape(X_last1year_scaled, (X_last1year_scaled.shape[0], 1, X_last1year_scaled.shape[1]))
X_last2year_reshaped = np.reshape(X_last2year_scaled, (X_last2year_scaled.shape[0], 1, X_last2year_scaled.shape[1]))
X_last3year_reshaped = np.reshape(X_last3year_scaled, (X_last3year_scaled.shape[0], 1, X_last3year_scaled.shape[1]))

# Combine last1year and last2year features along the time step dimension
X_combined_reshaped = np.concatenate((X_last3year_reshaped, X_last2year_reshaped, X_last1year_reshaped), axis=1)
#First use data from the past two years (X_last3year_reshaped), then data from the past years 

# Print data structure
print("Shape of X_last1year_scaled:", X_last1year_scaled.shape)
print("Shape of X_last2year_scaled:", X_last2year_scaled.shape)
print("Shape of X_last3year_scaled:", X_last3year_scaled.shape)
print("Shape of X_combined_reshaped:", X_combined_reshaped.shape)



# Split into train, validation, and test sets
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(X_combined_reshaped, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42) # 0.8 * 0.25 = 0.2

print("Shape of X_train:", X_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of X_test:", X_test.shape)


from imblearn.under_sampling import RandomUnderSampler

# Flatten X_train
X_train_flat = X_train.reshape(X_train.shape[0], -1)
# Apply undersampling only on the training set
undersampler = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train_flat, y_train)
print("Shape of X_train_resampled:", X_train_resampled.shape)
# Reshape resampled training data to match LSTM input shape
X_train_resampled = X_train_resampled.reshape(-1, 3, X_train.shape[2])
# Check the shape of resampled data and target variable
print("Shape of X_train_resampled:", X_train_resampled.shape)



from keras.layers import LSTM, Dense, Dropout, Input
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Reshape, Conv1D, MaxPooling1D, Flatten, ZeroPadding1D



#Define input layer
inputs = Input(shape=(3, X_train_resampled.shape[2]))
# Reshape input to fit Conv1D layer
inputs_reshaped = Reshape((X_train_resampled.shape[2], 3))(inputs)  
# Define CNN-LSTM model
cnn_layer = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs_reshaped)
# Add Padding1D layer
padded_layer = ZeroPadding1D(padding=2)(cnn_layer)  #Consider flatten being divisible by the LSTM time step, so add padding
pooling_layer = MaxPooling1D(pool_size=2)(padded_layer)
flattened_layer = Flatten()(pooling_layer)
print("Shape of flattened_layer:", flattened_layer.shape)
reshaped_layer = Reshape((3, -1))(flattened_layer)  # Reshape the output to fit LSTM input  #The input shape expected by the LSTM layer is (timesteps, features)
lstm_layer = LSTM(100)(reshaped_layer)   
dropout_layer = Dropout(0.2)(lstm_layer)
output_layer = Dense(1, activation='sigmoid')(dropout_layer)
# Define the model
model = Model(inputs=inputs, outputs=output_layer)
# Compile model
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
history = model.fit(X_train_resampled, y_train_resampled, epochs=100, batch_size=16, validation_data=(X_val, y_val), verbose=2)

## Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print("test loss:", loss)
print("Test accuracy:", accuracy)



Shape of X_last1year: (6298, 40)
Shape of X_last2year: (6298, 40)
Shape of X_last3year: (6298, 40)
Shape of y: (6298,)
Shape of X_last1year_scaled: (6298, 40)
Shape of X_last2year_scaled: (6298, 40)
Shape of X_last3year_scaled: (6298, 40)
Shape of X_combined_reshaped: (6298, 3, 40)
Shape of X_train: (3778, 3, 40)
Shape of X_val: (1260, 3, 40)
Shape of X_test: (1260, 3, 40)
Shape of X_train_resampled: (658, 120)
Shape of X_train_resampled: (658, 3, 40)
Shape of flattened_layer: (None, 1344)
Epoch 1/100
42/42 - 3s - 78ms/step - accuracy: 0.6474 - loss: 0.6332 - val_accuracy: 0.5984 - val_loss: 0.6930
Epoch 2/100
42/42 - 0s - 10ms/step - accuracy: 0.7128 - loss: 0.5597 - val_accuracy: 0.7095 - val_loss: 0.5123
Epoch 3/100
42/42 - 0s - 10ms/step - accuracy: 0.7204 - loss: 0.5348 - val_accuracy: 0.7770 - val_loss: 0.4189
Epoch 4/100
42/42 - 0s - 10ms/step - accuracy: 0.7052 - loss: 0.5367 - val_accuracy: 0.6270 - val_loss: 0.5857
Epoch 5/100
42/42 - 0s - 10ms/step - accuracy: 0.7447 - loss:

In [18]:
# Make predictions on the test set
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate evaluation metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Calculate precision
precision = precision_score(y_test, y_pred)

# Calculate recall
recall = recall_score(y_test, y_pred)

# Calculate F1 score
f1 = f1_score(y_test, y_pred)

# Calculate micro F1 score
micro_f1 = f1_score(y_test, y_pred, average='micro')

# Calculate macro F1 score
macro_f1 = f1_score(y_test, y_pred, average='macro')

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_test, y_pred_prob)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation metrics
print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("F1 Score:", f1)
print("Micro F1 Score:", micro_f1)
print("Macro F1 Score:", macro_f1)
print("ROC AUC:", roc_auc)
print("Confusion Matrix:")
print(conf_matrix)


40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
Accuracy: 0.6444444444444445
Recall: 0.6938775510204082
Precision: 0.13991769547325103
F1 Score: 0.23287671232876714
Micro F1 Score: 0.6444444444444445
Macro F1 Score: 0.5007358768255407
ROC AUC: 0.7377586146334609
Confusion Matrix:
[[744 418]
 [ 30  68]]
